In [61]:
pwd

'/Users/kelly/metis_v3/Project_5'

In [62]:
from pymongo import MongoClient
client = MongoClient()
db = client.goodreads

In [214]:
import nltk
from nltk.corpus import stopwords
import gensim
import numpy as np
import pprint

In [64]:
import pickle

In [65]:
with open('removed_tags.pickle','rb') as read_file:
    removed_tags = pickle.load(read_file)

removed_tag_list = []

for tag in removed_tags:
    for word in tag:
        removed_tag_list.append(word)

stop_words = stopwords.words('english')

removed_words = removed_tag_list + stop_words

In [66]:
with open('goodreads_id_list.pickle','rb') as read_file:
    goodreads_id_list = pickle.load(read_file)

In [67]:
goodreads_id_list

[2767052,
 3,
 41865,
 4671,
 2657,
 11870085,
 5907,
 5107,
 960,
 1885,
 77203,
 13335037,
 5470,
 7613,
 48855,
 2429135,
 6148028,
 5,
 34,
 7260188,
 2,
 12232938,
 15881,
 6,
 136251,
 968,
 1,
 7624,
 18135,
 8442457,
 4667024,
 890,
 930,
 10818853,
 865,
 3636,
 100915,
 14050,
 13496,
 19501,
 28187,
 1934,
 10210,
 15931,
 4214,
 43641,
 19063,
 4381,
 49041,
 30119,
 256683,
 428263,
 113436,
 11,
 1162543,
 5129,
 37435,
 2956,
 24178,
 1618,
 22557272,
 119322,
 6185,
 10917,
 4981,
 18405,
 128029,
 22628,
 11735983,
 375802,
 18490,
 11588,
 1656001,
 99561,
 227443,
 14935,
 38709,
 5139,
 1381,
 157993,
 7445,
 1845,
 1953,
 7677,
 370493,
 32542,
 1617,
 6442769,
 21787,
 231804,
 6186357,
 1202,
 2998,
 320,
 5297,
 13536860,
 17245,
 5060378,
 11857408,
 7244,
 4137,
 19543,
 7126,
 6288,
 9418327,
 18710190,
 3473,
 9416,
 24280,
 10572,
 10441,
 15507958,
 168668,
 6900,
 2187,
 24583,
 18131,
 7763,
 38447,
 452306,
 7604,
 37442,
 5358,
 7937843,
 1420,
 234225

In [68]:
def find_booktags (ID):
    '''
    Finds booktag ID numbers associated with each goodread
    '''
    booktag_list = []
    for book in db.booktags.find({'goodreads_book_id': ID}):
        booktag_list.append(book['tag_id'])
    return booktag_list

In [69]:
def find_booktags_name(ID):
    '''
    Finds descriptive 'tags' for all goodreads books and looks up associated tag name
    '''
    booktag_list = []          
    for book in db.booktags.find({'goodreads_book_id':ID}):
        tagid = (book['tag_id'])
        for tag in db.tags.find({'tag_id':tagid}):
            try:
                booktag_list.append(tag['tag_name'].split('-'))
            except:
                booktag_list.append(str(tag['tag_name']).split('-'))
    #print(booktag_list)
    return(booktag_list)

In [70]:
def create_booktag_list(ID):
    book_word_list = []
    for tag in find_booktags_name(ID):
        for word in tag:
            if word not in removed_words:
                book_word_list.append(word)
    return book_word_list

In [71]:
#vector_list = [] #cannot create whole list of all words for all 10,000 books!!!!!!
def create_word_vecs(goodreads_ids):
 #   for bookid in goodreads_ids:
    book_word_list = create_booktag_list(goodreads_id)
    #    vector_list.append(book_word_list)
    return book_word_list

In [72]:
#Use Google Model so don't have to train

google_vec_file = 'GoogleNews-vectors-negative300.bin'
google_model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [73]:
def create_document_vector(words, google_model):
    good_words = []
    for word in words:
        # Words not in the original model will fail
        try:
            if google_model.wv[word] is not None:
                good_words.append(word)
        except:
            continue
    # If no words are in the original model
    if len(good_words) == 0:
        return None
    # Return the mean of the vectors for all the good words
    return google_model.wv[good_words].mean(axis=0)

In [74]:
def create_vectors(vector_list):
    for vec in vector_list:
        book_vec = create_document_vector(vec, google_model)
  #      book_vecs.append(book_vec)
    return book_vec

In [75]:
#Run Model

In [81]:
'''
This is my final function to run the model - DO NOT ERASE!!!!!!

'''
book_vecs = []
for ix in goodreads_id_list[:]:
    book_word_list = create_booktag_list(ix)
    book_vec = create_vectors(book_word_list)
    book_vecs.append(book_vec)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [80]:
'''
for i in goodreads_id_list[:2]:
    book_word_list = create_booktag_list(i)
    print (len(book_word_list))
    '''

'\nfor i in goodreads_id_list[:2]:\n    book_word_list = create_booktag_list(i)\n    print (len(book_word_list))\n    '

In [82]:
#Check

len(book_vecs)

10000

In [79]:
import pandas as pd
vector_df = pd.DataFrame(book_vecs)
vector_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.309896,0.188802,-0.093913,0.123861,-0.082804,-0.000570,-0.026520,-0.122070,0.040527,-0.027588,...,-0.018433,0.026001,-0.106232,0.117676,-0.058594,-0.257812,-0.063070,-0.094035,0.010417,0.244792
1,-0.153551,0.114421,0.055284,0.123779,0.053406,-0.028703,-0.120524,-0.035482,-0.035312,0.009366,...,0.087863,0.043118,-0.081965,0.118490,-0.086304,-0.160970,-0.142415,0.005900,-0.141466,0.201226
2,-0.103271,0.121918,0.129089,0.106628,0.031746,0.044189,-0.059448,-0.063110,0.050537,0.020828,...,0.171814,-0.145142,-0.197388,0.134888,-0.040649,-0.206665,-0.040161,0.010834,-0.081299,0.198730
3,-0.178296,0.198242,0.021045,0.071436,-0.018323,0.003271,-0.122510,-0.034570,-0.037805,0.040295,...,0.132275,-0.016138,-0.139941,0.082813,-0.046436,-0.197754,-0.133789,-0.017114,-0.168848,0.191309
4,0.106201,-0.092651,0.071899,0.049194,-0.087250,-0.021727,0.150391,-0.174377,0.113525,0.132690,...,0.028015,-0.053650,-0.156982,0.177979,-0.077087,0.023621,-0.144073,-0.097260,-0.054504,0.110931


In [86]:
import sys
sys.getsizeof(book_vecs)

87624

In [87]:
with open ('book_vecs.pickle', 'wb') as to_write:
    pickle.dump(book_vecs, to_write)

In [233]:
import copy
book_vec_copy = copy.copy(book_vecs)

In [122]:
len(book_vec_copy[0:837])

837

In [127]:
type(book_vec_copy[837])

numpy.ndarray

In [128]:
type(book_vec_copy[838])

NoneType

In [139]:
#Remove books that have no associated word tags

for ix, vec in enumerate(book_vec_copy[0:10000]):
    if vec is None:
        print (ix)

488
838
1371
1834
3122
3382
4044
4118
4190
4398
4420
4474
4515
4590
4664
4759
4825
4959
4979
5243
5372
5705
6069
6118
6912
7023
7241
7449
7605
7822
7936
7962
8402
8881
9007
9038
9077
9220
9242
9326
9569
9609
9729
9769
9828
9857
9966


In [145]:
def find_goodreads_title(ID):
    for book in db.books.find({'goodreads_book_id':ID}):
        return(book['original_title'])

In [148]:
book_titles = []
for goodreads_id in goodreads_id_list:
    book_title = find_goodreads_title(goodreads_id)
    book_titles.append(book_title)

In [154]:
book_titles

['The Hunger Games',
 "Harry Potter and the Philosopher's Stone",
 'Twilight',
 'The Great Gatsby',
 'To Kill a Mockingbird',
 'The Fault in Our Stars',
 'The Hobbit or There and Back Again',
 'The Catcher in the Rye',
 'Angels & Demons',
 'Pride and Prejudice',
 'The Kite Runner',
 'Divergent',
 'Nineteen Eighty-Four',
 'Animal Farm: A Fairy Story',
 'Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944',
 'Män som hatar kvinnor',
 'Catching Fire',
 'Harry Potter and the Prisoner of Azkaban',
 'The Fellowship of the Ring',
 'Mockingjay',
 'Harry Potter and the Order of the Phoenix',
 'The Lovely Bones',
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Deathly Hallows',
 'The Da Vinci Code',
 'Harry Potter and the Half-Blood Prince',
 'Lord of the Flies',
 'An Excellent conceited Tragedie of Romeo and Juliet',
 'Gone Girl',
 'The Help',
 'Of Mice and Men',
 'Memoirs of a Geisha',
 'Fifty Shades of Grey',
 'O Alquimista'

In [227]:
book_vec_copy[838]

In [235]:
for ix, vec in enumerate(book_vec_copy[0:10000]):
    if vec is None:
        book_vec_copy[ix] = (np.array(['NaN']*300))

In [236]:
for ix, vec in enumerate(book_vec_copy[0:10000]):
    if vec is None:
        print (ix)

In [237]:
vec_df = pd.DataFrame(book_vec_copy)

In [238]:
vec_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.309896,0.188802,-0.0939128,0.123861,-0.0828044,-0.000569661,-0.0265198,-0.12207,0.0405273,-0.0275879,...,-0.0184326,0.026001,-0.106232,0.117676,-0.0585938,-0.257812,-0.0630697,-0.0940348,0.0104167,0.244792
1,-0.153551,0.114421,0.0552843,0.123779,0.0534058,-0.0287035,-0.120524,-0.0354818,-0.0353122,0.00936551,...,0.0878635,0.0431179,-0.0819651,0.11849,-0.0863037,-0.16097,-0.142415,0.00590006,-0.141466,0.201226
2,-0.103271,0.121918,0.129089,0.106628,0.0317459,0.0441895,-0.0594482,-0.0631104,0.0505371,0.0208282,...,0.171814,-0.145142,-0.197388,0.134888,-0.0406494,-0.206665,-0.0401611,0.0108337,-0.0812988,0.19873
3,-0.178296,0.198242,0.0210449,0.0714355,-0.0183228,0.00327148,-0.12251,-0.0345703,-0.0378052,0.0402954,...,0.132275,-0.0161377,-0.139941,0.0828125,-0.0464355,-0.197754,-0.133789,-0.0171143,-0.168848,0.191309
4,0.106201,-0.0926514,0.0718994,0.0491943,-0.0872498,-0.0217266,0.150391,-0.174377,0.113525,0.13269,...,0.0280151,-0.0536499,-0.156982,0.177979,-0.0770874,0.0236206,-0.144073,-0.0972595,-0.0545044,0.110931
5,-0.179901,0.0880432,0.0504557,0.165446,-0.0650431,0.0303497,-0.0646566,-0.047526,-0.0535482,-0.00084432,...,0.0561523,-0.031189,-0.0347087,0.114827,-0.0566406,-0.135254,-0.101562,-0.0296631,-0.133545,0.152334
6,-0.168811,0.0969727,0.0492188,0.137939,-0.0901672,0.0105042,-0.0994141,-0.0581543,-0.0155334,0.0511292,...,0.106934,0.0516113,-0.0830078,0.0724121,-0.0715332,-0.181982,-0.14541,-0.0353882,-0.171021,0.104797
7,-0.168811,0.0969727,0.0492188,0.137939,-0.0901672,0.0105042,-0.0994141,-0.0581543,-0.0155334,0.0511292,...,0.106934,0.0516113,-0.0830078,0.0724121,-0.0715332,-0.181982,-0.14541,-0.0353882,-0.171021,0.104797
8,-0.13205,0.137065,-0.00738525,0.143555,-0.0277913,0.050649,-0.136149,-0.00805664,-0.0909831,0.0671488,...,0.0648193,-0.0298462,-0.0994466,0.11672,-0.0555827,-0.129395,-0.0832723,-0.0104574,-0.147217,0.0989787
9,-0.144999,0.119995,0.0611572,0.115356,-0.106934,0.0362193,-0.149902,-0.099528,-0.0692342,0.00741577,...,0.0643311,0.031189,-0.119367,0.0597331,-0.0182088,-0.122803,-0.127197,-0.0496623,-0.134196,0.151377


In [263]:
book_vec_df = pd.concat([pd.DataFrame(book_titles).rename(columns = {0:'Title'}), vec_df], axis = 1)

In [264]:
book_vec_df

,Title,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,The Hunger Games,-0.309896,0.188802,-0.0939128,0.123861,-0.0828044,-0.000569661,-0.0265198,-0.12207,0.0405273,...,-0.0184326,0.026001,-0.106232,0.117676,-0.0585938,-0.257812,-0.0630697,-0.0940348,0.0104167,0.244792
1,Harry Potter and the Philosopher's Stone,-0.153551,0.114421,0.0552843,0.123779,0.0534058,-0.0287035,-0.120524,-0.0354818,-0.0353122,...,0.0878635,0.0431179,-0.0819651,0.11849,-0.0863037,-0.16097,-0.142415,0.00590006,-0.141466,0.201226
2,Twilight,-0.103271,0.121918,0.129089,0.106628,0.0317459,0.0441895,-0.0594482,-0.0631104,0.0505371,...,0.171814,-0.145142,-0.197388,0.134888,-0.0406494,-0.206665,-0.0401611,0.0108337,-0.0812988,0.19873
3,The Great Gatsby,-0.178296,0.198242,0.0210449,0.0714355,-0.0183228,0.00327148,-0.12251,-0.0345703,-0.0378052,...,0.132275,-0.0161377,-0.139941,0.0828125,-0.0464355,-0.197754,-0.133789,-0.0171143,-0.168848,0.191309
4,To Kill a Mockingbird,0.106201,-0.0926514,0.0718994,0.0491943,-0.0872498,-0.0217266,0.150391,-0.174377,0.113525,...,0.0280151,-0.0536499,-0.156982,0.177979,-0.0770874,0.0236206,-0.144073,-0.0972595,-0.0545044,0.110931
5,The Fault in Our Stars,-0.179901,0.0880432,0.0504557,0.165446,-0.0650431,0.0303497,-0.0646566,-0.047526,-0.0535482,...,0.0561523,-0.031189,-0.0347087,0.114827,-0.0566406,-0.135254,-0.101562,-0.0296631,-0.133545,0.152334
6,The Hobbit or There and Back Again,-0.168811,0.0969727,0.0492188,0.137939,-0.0901672,0.0105042,-0.0994141,-0.0581543,-0.0155334,...,0.106934,0.0516113,-0.0830078,0.0724121,-0.0715332,-0.181982,-0.14541,-0.0353882,-0.171021,0.104797
7,The Catcher in the Rye,-0.168811,0.0969727,0.0492188,0.137939,-0.0901672,0.0105042,-0.0994141,-0.0581543,-0.0155334,...,0.106934,0.0516113,-0.0830078,0.0724121,-0.0715332,-0.181982,-0.14541,-0.0353882,-0.171021,0.104797
8,Angels & Demons,-0.13205,0.137065,-0.00738525,0.143555,-0.0277913,0.050649,-0.136149,-0.00805664,-0.0909831,...,0.0648193,-0.0298462,-0.0994466,0.11672,-0.0555827,-0.129395,-0.0832723,-0.0104574,-0.147217,0.0989787
9,Pride and Prejudice,-0.144999,0.119995,0.0611572,0.115356,-0.106934,0.0362193,-0.149902,-0.099528,-0.0692342,...,0.0643311,0.031189,-0.119367,0.0597331,-0.0182088,-0.122803,-0.127197,-0.0496623,-0.134196,0.151377


In [265]:
with open ('book_vec_df.pickle', 'wb') as to_write:
    pickle.dump(book_vec_df, to_write)